In [10]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Configura los detalles de conexión
dbname = "fast_SQL_python"  # Reemplaza con el nombre que desees para la nueva base de datos
user = "postgres"          # Reemplaza con tu nombre de usuario
password = "1qaz0OKM"   # Reemplaza con tu contraseña
host = "127.0.0.1"           # Reemplaza con la dirección del servidor PostgreSQL

In [3]:
# verifica que la conexion con la base de datos sea exitosa e imprime un mensaje, adicional crea un cursor

cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    cur=conn.cursor()
    print("Conexión exitosa a la base de datos.")
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Conexión exitosa a la base de datos.


## Crear Tabla

In [37]:
# verifica que la conexion con la base de datos sea exitosa e imprime un mensaje, adicional crea un cursor

cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--

    cur.execute("DROP TABLE IF EXISTS public.wine_rating")

    cur.execute("""
                CREATE TABLE IF NOT EXISTS wine_rating (
                    id serial NOT NULL,
                    name character varying(255) NOT NULL,
                    grape character varying(255) NOT NULL,
                    region character varying(255) NOT NULL,
                    variety character varying(255) NOT NULL,
                    rating integer NOT NULL,
                    notes character varying(255) NOT NULL
                )
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

## Definir Primary Key en Tabla Existente

In [38]:
# definir prymary key en una tabla y columna existente

cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--
    
    cur.execute("""
                ALTER TABLE IF EXISTS public.wine_rating
                    ADD CONSTRAINT sdfdsfdsfdsfd PRIMARY KEY (id);
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

## Definir Foreing Key en Tablas Existentes

In [39]:
# definir foreing key en una tabla y columna existente, tenga en cuenta que las tablas y columnas deben coincidir 

cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--
    
    cur.execute("""
                ALTER TABLE IF EXISTS wine_rating
                    ADD CONSTRAINT dfgdfgdfdfgdf FOREIGN KEY (id)
                    REFERENCES public.wine_rating (name) MATCH FULL
                    ON UPDATE CASCADE
                    ON DELETE CASCADE;
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Error al conectar a la base de datos: no hay restricción unique que coincida con las columnas dadas en la tabla referida «wine_rating»



## Crear una Particion de Tablas

In [46]:
# definir foreing key en una tabla y columna existente, tenga en cuenta que las tablas y columnas deben coincidir 

cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--

    cur.execute("""
                CREATE TABLE IF NOT EXISTS part_pasajeros_201001 PARTITION OF pasajeros
                FOR VALUES FROM ('2010-01-01') TO ('2010-01-31')
                """)
    
    cur.execute("""
                CREATE TABLE IF NOT EXISTS pasajeros (
                    id serial NOT NULL,
                    pasajero character varying(100),
                    fecha date
                ) PARTITION BY RANGE (fecha)
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

## Crear Roles desde Python

In [4]:
# Creacion de usuario para solo consulta, no puede crear, eliminar o modificar tablas
cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--

    cur.execute("""
                CREATE ROLE "ILBERT" WITH
                    LOGIN
                    NOSUPERUSER
                    NOCREATEDB
                    NOCREATEROLE
                    INHERIT
                    NOREPLICATION
                    CONNECTION LIMIT -1
                    PASSWORD '123456';
                COMMENT ON ROLE "ILBERT" IS 'usuario de consulta'
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

## Asignacion de Roles a una Tabla

In [9]:
cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--

    cur.execute("""
                GRANT INSERT, UPDATE, SELECT ON TABLE pasajeros TO "ILBERT";
                """)

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

## Cargar Filas

In [11]:
df = pd.read_json('sample_data\wine-ratings.json')

In [15]:
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

# 3. Cargar datos en la tabla existente en PostgreSQL
table_name = "wine_rating"  # Reemplaza con el nombre de tu tabla en PostgreSQL

df.to_sql(table_name, engine, if_exists='append', index=True)

# Cierra la conexión
engine.dispose()

print(f'Datos cargados en la tabla {table_name}')

Datos cargados en la tabla wine-rating


## Realizar Consultas

In [58]:
cur=None

try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    cur=conn.cursor()

    ##--

    # Ejecuta una consulta SQL
    consulta = "SELECT * FROM wine_rating;"  # Reemplaza 'mi_tabla' con el nombre de tu tabla
    df_consulta = pd.read_sql_query(consulta, conn).set_index('index')  # Carga los resultados en un DataFrame

    # Imprime el DataFrame
    df_consulta

    conn.commit()
    ##--
except psycopg2.Error as e:
    print("Error al conectar a la base de datos:", e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

C:\Users\Alarcon Ilbert\AppData\Local\Temp\ipykernel_11028\2877578232.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_consulta = pd.read_sql_query(consulta, conn).set_index('index')  # Carga los resultados en un DataFrame


In [59]:
df_consulta

,name,grape,region,variety,rating,notes
index,,,,,,
0,Laurenz V Charming Gruner Veltliner 2013,None,"Kamptal, Austria",White Wine,90,Aromas of ripe apples and a typical Veltliner ...
1,Laurenz V Charming Gruner Veltliner 2014,None,"Kamptal, Austria",White Wine,90,Aromas of ripe apples and a typical Veltliner ...
2,Laurenz V Singing Gruner Veltliner 2007,None,Austria,White Wine,90,"A very attractive fruit bouquet yields apple, ..."
3,Laurenz V Singing Gruner Veltliner 2010,None,Austria,White Wine,88,"A very attractive fruit bouquet yields apple, ..."
4,Laurenz V Singing Gruner Veltliner 2011,None,Austria,White Wine,88,"A very attractive fruit bouquet yields apple, ..."
...,...,...,...,...,...,...
775,Lewis Cellars Syrah Ethan's Blend 2005,None,California,Red Wine,92,"Now 7 years old, Alec's younger brother Ethan ..."
776,Lewis Cellars Syrah Ethan's Blend 2014,None,California,Red Wine,93,"Born in 2000, everything about Ethan and his n..."
777,Lewis Cellars Syrah Ethan's Blend 2009,None,California,Red Wine,94,"Positioned between brothers, Alec and Mason, E..."
